## Librerias y Datos

In [3]:
import os
import mne
import numpy as np
import pandas as pd
from scipy.signal import welch

# 1. Configuración inicial
path = "Datos/Subjects/S2"
fs = 256  # Frecuencia de muestreo
window_duration = 5  # Duración de ventana en segundos
nperseg = fs * window_duration  # Muestras por ventana
baseline_duration = 30  # Línea base: primeros 30 segundos


## Cargar Datos

In [4]:
all_data = pd.DataFrame()

for idx, filename in enumerate(os.listdir(path)):
    if filename.endswith(".set"):
        raw = mne.io.read_raw_eeglab(os.path.join(path, filename), preload=True)
        channels = raw.ch_names[:32]
        data, _ = raw[channels, :]
        
        df = pd.DataFrame(data.T, columns=channels)
        df['ID'] = idx + 1
        all_data = pd.concat([all_data, df], ignore_index=True)
display(all_data.head())

/home/ianedmosz/anaconda3/lib/python3.12/site-packages/pymatreader/utils.py:168: UserWarning: Complex objects (like classes) are not supported. They are imported on a best effort base but your mileage will vary.
  warn(
/home/ianedmosz/anaconda3/lib/python3.12/site-packages/pymatreader/utils.py:168: UserWarning: Complex objects (like classes) are not supported. They are imported on a best effort base but your mileage will vary.
  warn(


,FP1,FP2,F7,F3,Fz,F4,F8,FT10,FT9,T7,...,T8,P7,P3,Pz,P4,P8,O1,Oz,O2,ID
0,0.000698,0.000699,0.000744,0.000743,0.000741,0.000759,0.000740,0.000746,0.000671,-0.000780,...,0.000416,-0.001344,-0.000460,-0.000396,-0.001373,-0.000453,-0.001384,-0.001347,-0.001341,1
1,0.001329,0.001335,0.001399,0.001397,0.001413,0.001406,0.001419,0.001420,0.001346,-0.001642,...,0.000851,-0.002614,-0.000821,-0.000689,-0.002500,-0.000777,-0.002566,-0.002629,-0.002607,1
2,0.000974,0.000980,0.001028,0.001026,0.001048,0.001028,0.001057,0.001054,0.001024,-0.001331,...,0.000677,-0.001949,-0.000558,-0.000452,-0.001778,-0.000523,-0.001849,-0.001964,-0.001943,1
3,-0.000183,-0.000182,-0.000180,-0.000179,-0.000172,-0.000177,-0.000167,-0.000170,-0.000153,0.000073,...,-0.000049,0.000318,0.000163,0.000155,0.000385,0.000151,0.000377,0.000318,0.000319,1
4,-0.000509,-0.000511,-0.000523,-0.000522,-0.000526,-0.000520,-0.000525,-0.000524,-0.000514,0.000589,...,-0.000301,0.000967,0.000315,0.000269,0.000945,0.000304,0.000972,0.000975,0.000967,1


## Seleccion de sujeto  

In [5]:
df_id2 = all_data[all_data['ID'] == 1].copy()
nombres_canales = [col for col in df_id2.columns if col != 'ID']
data = df_id2[nombres_canales].values.T  # shape: (n_canales, n_muestras)


In [6]:
frontal_theta = ['Fz', 'F3', 'F4']
parietal_alpha = ['Pz', 'P3', 'P4']
prefrontal_beta = ['FP1', 'FP2']

baseline_samples = baseline_duration * fs
freqs_baseline, psd_baseline = welch(
    data[:, :baseline_samples], 
    fs=fs, 
    nperseg=nperseg,
    axis=1
)

# Obtener índices de canal
idx_frontal = [nombres_canales.index(ch) for ch in frontal_theta if ch in nombres_canales]
idx_parietal = [nombres_canales.index(ch) for ch in parietal_alpha if ch in nombres_canales]
idx_prefrontal = [nombres_canales.index(ch) for ch in prefrontal_beta if ch in nombres_canales]

# Obtener índices de frecuencia para línea base
idx_theta_base = np.where((freqs_baseline >= 4) & (freqs_baseline <= 8))[0]
idx_alpha_base = np.where((freqs_baseline >= 8) & (freqs_baseline <= 12))[0]
idx_beta_base = np.where((freqs_baseline >= 12) & (freqs_baseline <= 30))[0]

# Calcular potencias base
theta_baseline = psd_baseline[idx_frontal][:, idx_theta_base].mean()
alpha_baseline = psd_baseline[idx_parietal][:, idx_alpha_base].mean()
beta_baseline = psd_baseline[idx_prefrontal][:, idx_beta_base].mean()

engagement = []
n_muestras = data.shape[1]
n_ventanas = int(np.ceil(n_muestras / nperseg))

for i in range(n_ventanas):
    start = i * nperseg
    end = min((i + 1) * nperseg, n_muestras)
    segmento = data[:, start:end]

    if segmento.shape[1] < fs:  # Mínimo 1 segundo para análisis
        continue

    freqs, psd = welch(segmento, fs=fs, nperseg=min(nperseg, segmento.shape[1]), axis=1)

    # Índices por ventana
    idx_theta = np.where((freqs >= 4) & (freqs <= 8))[0]
    idx_alpha = np.where((freqs >= 8) & (freqs <= 12))[0]
    idx_beta = np.where((freqs >= 12) & (freqs <= 30))[0]

    if len(idx_theta) == 0 or len(idx_alpha) == 0 or len(idx_beta) == 0:
        continue

    # Normalización por línea base
    theta_norm = psd[idx_frontal][:, idx_theta].mean() / theta_baseline
    alpha_norm = psd[idx_parietal][:, idx_alpha].mean() / alpha_baseline
    beta_norm = psd[idx_prefrontal][:, idx_beta].mean() / beta_baseline

    # Índices de carga cognitiva
    eng = theta_norm / alpha_norm if alpha_norm > 0 else np.nan
    eng_beta = beta_norm / (alpha_norm + theta_norm) if (alpha_norm + theta_norm) > 0 else np.nan
    fatigue_index = alpha_norm / theta_norm if theta_norm > 0 else np.nan

    engagement.append({
        'ventana': i + 1,
        'inicio_seg': start/fs,
        'fin_seg': end/fs,
        'theta_frontal_norm': theta_norm,
        'alpha_parietal_norm': alpha_norm,
        'beta_prefrontal_norm': beta_norm,
        'engagement_Index': eng_beta,
        'task_load_index': eng,
        'fatigue_index': fatigue_index
    })

df_engagement = pd.DataFrame(engagement)
print(f"\nTotal de ventanas: {len(df_engagement)}")
print(df_engagement.head(10))
print(f"\nPromedio TLI normalizado: {df_engagement['task_load_index'].mean():.2f}")
print(f"Promedio EBI normalizado: {df_engagement['engagement_Index'].mean():.2f}")

df_engagement.to_csv("engagement_normalizado.csv", index=False)



Total de ventanas: 100
   ventana  inicio_seg  fin_seg  theta_frontal_norm  alpha_parietal_norm  \
0        1         0.0      5.0            1.214355             2.768411   
1        2         5.0     10.0            0.137540             0.105555   
2        3        10.0     15.0            0.426060             0.461560   
3        4        15.0     20.0            0.275548             0.383946   
4        5        20.0     25.0            0.112397             0.070527   
5        6        25.0     30.0            1.168416             0.989202   
6        7        30.0     35.0            0.950476             1.030528   
7        8        35.0     40.0            0.164568             0.276724   
8        9        40.0     45.0            1.372405             1.203963   
9       10        45.0     50.0            0.181860             0.240599   

   beta_prefrontal_norm  engagement_Index  task_load_index  fatigue_index  
0              2.324659          0.583680         0.438647     